In [34]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [35]:
train.shape

(891, 12)

In [36]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [37]:
# train = train.drop("Cabin", axis=1)
# test = test.drop("Cabin", axis=1)


In [38]:
train = train.drop("PassengerId", axis=1)
passenger_id = test["PassengerId"]
test = test.drop("PassengerId", axis=1)

In [39]:
train = train.drop("Ticket", axis=1)
test = test.drop("Ticket", axis=1)


In [40]:
train["Pclass"].unique()

array([3, 1, 2], dtype=int64)

In [41]:
train["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [42]:
# train["Age"] = train.groupby("Pclass")["Age"].transform(lambda x: x.fillna(x.mode()[0]))

In [43]:
import re

def extract_surname(name):
    match = re.match(r'([^,]+)', name)
    if match:
        return match.group(1)
    return ''

# Apply the function to the 'Name' column
train['Name'] = train['Name'].apply(extract_surname)
test['Name'] = test['Name'].apply(extract_surname)

In [44]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
train["Cabin"] = label_encoder.fit_transform(train["Cabin"])
test["Cabin"] = label_encoder.fit_transform(test["Cabin"])
train["Sex"] = label_encoder.fit_transform(train["Sex"])
train["Embarked"] = label_encoder.fit_transform(train["Embarked"])
train["Name"] = label_encoder.fit_transform(train["Name"])
test["Sex"] = label_encoder.fit_transform(test["Sex"])
test["Embarked"] = label_encoder.fit_transform(test["Embarked"])
test["Name"] = label_encoder.fit_transform(test["Name"])

In [45]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,73,1,22.0,1,0,7.2500,147,2
1,1,1,136,0,38.0,1,0,71.2833,81,0
2,1,3,251,0,26.0,0,0,7.9250,147,2
3,1,1,198,0,35.0,1,0,53.1000,55,2
4,0,3,11,1,35.0,0,0,8.0500,147,2
...,...,...,...,...,...,...,...,...,...,...
886,0,2,406,1,27.0,0,0,13.0000,147,2
887,1,1,221,0,19.0,0,0,30.0000,30,2
888,0,3,293,0,NaN,1,2,23.4500,147,2
889,1,1,52,1,26.0,0,0,30.0000,60,0


In [46]:
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=5)

# def impute_group(group):
#     return pd.DataFrame(imputer.fit_transform(group), columns=group.columns, index=group.index)

# train_imputed = train.groupby('Pclass').apply(impute_group)
# train_imputed = test.groupby('Pclass').apply(impute_group)

In [47]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
X = train.drop("Survived", axis=1)
y = train["Survived"]

classifier = HistGradientBoostingClassifier()

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage
    'max_iter': [100, 200, 300],         # Number of boosting iterations
    'max_depth': [None, 3, 5, 7],        # Maximum depth of the trees
    'min_samples_leaf': [10, 20, 50],    # Minimum samples per leaf
    'l2_regularization': [0.0, 1.0],     # L2 regularization strength
    'max_bins': [50, 100]               # Number of bins for histograms
}

# Set up Grid Search with cross-validation
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, 
                           cv=5,  # Number of cross-validation folds
                           n_jobs=-1,  # Number of parallel jobs to run (-1 means using all processors)
                           verbose=2,  # Controls the verbosity of the output
                           scoring='roc_auc')  # Metric to optimize

# Fit Grid Search to the data
grid_search.fit(X, y)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=-1,
             param_grid={'l2_regularization': [0.0, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_bins': [50, 100], 'max_depth': [None, 3, 5, 7],
                         'max_iter': [100, 200, 300],
                         'min_samples_leaf': [10, 20, 50]},
             scoring='roc_auc', verbose=2)

In [48]:
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(test)

In [49]:
y_pred = pd.Series(y_pred, name="Survived")

In [50]:
y_pred

0      0
1      0
2      0
3      0
4      0
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [51]:
to_some_file = pd.concat([passenger_id,pd.Series(y_pred)], axis=1)

In [52]:
to_some_file.to_csv("gender_predictions_hist.csv", index=False)